# Rings segmentation

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam

from dlia_tools.u_net import u_net
from dlia_tools.keras_image2image import DeadLeavesWithSegmGenerator
from dlia_tools.keras_custom_loss import jaccard2_loss
from dlia_tools.random_image_generator import AdditiveGaussianNoise
from dlia_tools.random_image_generator import ROG_disks, ROG_rings, RandomPosGenUniform, RandomIntGenUniform

# *** Params
# architecture params
nb_filters_0 = 6
sigma_noise = 0.01

# compile params
opt_name = 'rmsprop'  # bon:adadelta; sgd, rmsprop, adagrad, adam
loss_func = jaccard2_loss  # mse, mae, binary_crossentropy

# fit params
batch_size = 32
nb_train_samples = 10000
nb_epoch = 5
nb_val_samples = 1000
verbose = 1

# SGD parameters
lr = 0.1  # 0.01
decay = 1e-6  # 1e-6
momentum = 0.9  # 0.9
nesterov = True

# ****  input data generator
img_rows, img_cols = 32, 32
img_channels = 1
gauss_n_std = 40
nb_obj_l = 1
nb_obj_h = 4
r1_disk_l = 2
r1_disk_h = 4
r1_ring_l = 4
r1_ring_h = 8
grey_l = 20
grey_h = 200
norm = 255  # normalization constant

# List of random objects that will be used to build the image
l_rog = [
    ROG_rings(
        RandomIntGenUniform(nb_obj_l, nb_obj_h),
        RandomPosGenUniform(img_rows, img_cols),
        RandomIntGenUniform(r1_ring_l, r1_ring_h),
        RandomIntGenUniform(grey_l, grey_h),
        gt=1,
        rad_ratio=0.5),
    ROG_disks(
        RandomIntGenUniform(3 * nb_obj_l, 3 * nb_obj_h),
        RandomPosGenUniform(img_rows, img_cols),
        RandomIntGenUniform(r1_ring_l, r1_ring_h),
        RandomIntGenUniform(grey_l, grey_h),
        gt=0),
]

noise = AdditiveGaussianNoise(gauss_n_std)
datagen = DeadLeavesWithSegmGenerator(
    img_rows,
    img_cols,
    l_rog,
    noise,
    background_val=0,
    shuffle=False,
    norm=norm)
datagen_val = DeadLeavesWithSegmGenerator(
    img_rows,
    img_cols,
    l_rog,
    noise,
    background_val=0,
    shuffle=False,
    norm=norm)

if opt_name == "sgd":
    opt = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
elif opt_name == "rmsprop":
    opt = RMSprop()
elif opt_name == "adagrad":
    opt = Adagrad()
elif opt_name == "adadelta":
    opt = Adadelta()
elif opt_name == "adam":
    opt = Adam(lr=1e-5)
else:
    raise NameError("Wrong optimizer name")

# ****  deep learning model
shape = (img_rows, img_cols, img_channels)
model = u_net(shape, nb_filters_0, sigma_noise=sigma_noise)
# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])
# from this point on, all prints go to log file

# ****  train
model.compile(loss=loss_func, optimizer=opt)
print(model.summary())

print("Generating images")
(X_train, Y_train) = next(datagen.flow(batch_size=nb_train_samples))
(X_val0, Y_val0) = next(datagen_val.flow(batch_size=nb_val_samples))
print("Generation finished")

print(X_val0.shape, Y_val0.shape, X_train.shape, Y_train.shape)

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(X_val0, Y_val0),
                    shuffle=True,
                    verbose=verbose)

# **** #####################################"
print("Best validation loss: %.5f" % (np.min(history.history['val_loss'])))
print("at: %d" % np.argmin(history.history['val_loss']))

plt.plot(history.epoch, history.history['loss'], label='train')
plt.plot(history.epoch, history.history['val_loss'], label='val')
plt.title('Training performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.ylim(0.0, 0.9)
plt.show()

# Displaying intermediate layers
# from utils_for_macula_detection import display_all_layer_filters
# import matplotlib.pyplot as plt
# layer_name = None
# #layers_to_visu = ["conv0", "conv1", "conv2", "conv3", "conv4", ]
# layers_to_visu = ["conv%s"%(i) for i in range(layers_nb)]
# img_iter = datagen.flow(1)
# img = next(img_iter)[0][0,0]
# if layer_name is not None:
#     display_all_layer_filters(model, layer_dict, layer_name, img)
# if layers_to_visu is not None:
#     for layer_n in layers_to_visu: display_all_layer_filters(model, layer_dict, layer_n, img)
